Developed a four-agent AI system using Groq’s LLaMA 3.3 model and LlamaIndex-based RAG pipeline for automated financial intelligence. The system includes a Research Analyst Agent for data gathering and analysis, an Earnings Report Agent for retrieval-augmented insights, a Stock Report Generator for automated summaries, and an LLM-as-a-Judge for evaluating output reliability. Combined advanced multi-agent coordination, retrieval-augmented generation, and financial domain reasoning for robust, end-to-end research automation.

# **Agent 1: Financial Research Analyst Agent**


### Install & Import Required Packages

In [1]:
!pip install -q groq duckduckgo-search lxml_html_clean agno
!pip install -q newspaper4k lxml_html_clean
!pip install -q ddgs

In [2]:
# Imports after successful installation
from textwrap import dedent
from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.newspaper4k import Newspaper4kTools

import os
from google.colab import userdata

### Set API Keys

In [3]:
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['PHI_API_KEY'] = userdata.get('PHI_API_KEY')

print("API keys have been set!")

API keys have been set!


### The Research Analyst Agent With Advanced Capabilities

In [4]:
research_agent = Agent(
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[DuckDuckGoTools(), Newspaper4kTools()],
    description=dedent("""\
        You are an elite research analyst in the financial services domain.
        Your expertise encompasses:

        - Deep investigative financial research and analysis
        - fact-checking and source verification
        - Data-driven reporting and visualization
        - Expert interview synthesis
        - Trend analysis and future predictions
        - Complex topic simplification
        - Ethical practices
        - Balanced perspective presentation
        - Global context integration\
    """),
    instructions=dedent("""\
        1. Research Phase
           - Search for 5 authoritative sources on the topic
           - Prioritize recent publications and expert opinions
           - Identify key stakeholders and perspectives

        2. Analysis Phase
           - Extract and verify critical information
           - Cross-reference facts across multiple sources
           - Identify emerging patterns and trends
           - Evaluate conflicting viewpoints

        3. Writing Phase
           - Craft an attention-grabbing headline
           - Structure content in Financial Report style
           - Include relevant quotes and statistics
           - Maintain objectivity and balance
           - Explain complex concepts clearly

        4. Quality Control
           - Verify all facts and attributions
           - Ensure narrative flow and readability
           - Add context where necessary
           - Include future implications
    """),
    expected_output=dedent("""\
        # {Compelling Headline}

        ## Executive Summary
        {Concise overview of key findings and significance}

        ## Background & Context
        {Historical context and importance}
        {Current landscape overview}

        ## Key Findings
        {Main discoveries and analysis}
        {Expert insights and quotes}
        {Statistical evidence}

        ## Impact Analysis
        {Current implications}
        {Stakeholder perspectives}
        {Industry/societal effects}

        ## Future Outlook
        {Emerging trends}
        {Expert predictions}
        {Potential challenges and opportunities}

        ## Expert Insights
        {Notable quotes and analysis from industry leaders}
        {Contrasting viewpoints}

        ## Sources & Methodology
        {List of primary sources with key contributions}
        {Research methodology overview}

        ---
        Research conducted by Financial Agent
        Credit Rating Style Report
        Published: {current_date}
        Last Updated: {current_time}\
    """),
    markdown=True,
)

# User Prompt 1
research_agent.print_response("Analyze the current state and future implications \
                              of artificial intelligence in Finance", stream=True)


Output()

# **Agent 2: Agentic RAG on the Apple Earnings Report**

### Install & Import Required Packages

In [5]:
!pip install llama-index llama-index-vector-stores-chroma llama-index-llms-groq llama-index-embeddings-huggingface -q

In [6]:
import os
import time
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

### Part 1: Load and process the document

In [7]:
print("Loading document...")
reader = SimpleDirectoryReader(input_files=["Apple Q4 2025 Earnings Call Transcript.pdf"])
documents = reader.load_data()
print(f"✓ Loaded {len(documents)} document(s)")

Loading document...
✓ Loaded 19 document(s)


### Split the document into chunks of max 1024 tokens (sentences)

In [8]:
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

### Part 2: Configure LLM and embedding model

In [9]:
Settings.llm = Groq(model="llama-3.1-8b-instant", timeout=60)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
print("✓ Models configured successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✓ Models configured successfully!


### Part 3: Create the indexes

In [10]:
print("Creating indexes...")
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)
print("✓ Indexes created successfully!")

Creating indexes...
✓ Indexes created successfully!


### Create query engines from indexes

In [11]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=False,
)

vector_query_engine = vector_index.as_query_engine()

### Part 4: Wrap query engines into tools

In [12]:
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions about the Earnings report.",
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for finding specific facts and details from the Earnings report.",
)

### Part 5: Create router query engine

In [13]:
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[summary_tool, vector_tool],
    verbose=True
)

### Part 6: Create router query engine to route between tools

In [14]:
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[summary_tool, vector_tool],
    verbose=False,
)

print("✓ Agentic RAG system ready!")

✓ Agentic RAG system ready!


### Part 7: Define function to ask questions with delay handling

In [15]:
def ask_question(question):
    print(f"\nQuestion: {question}")
    print("=" * 50)
    response = query_engine.query(question)
    print(f"Answer: {response}")
    return response

### Part 8: Test the system with example questions and delays to avoid rate limits

In [16]:
print("\nTesting the Agentic RAG System...")
print("=" * 60)

test_questions = [
    "What was the total revenue for the fourth quarter of fiscal 2025, and what was the year-over-year growth percentage?",
    "What is Apple's revenue growth forecast for the December quarter, and what is the primary reason for the expected strong performance in iPhone sales?"]

for i, question in enumerate(test_questions):
    print(f"\nTest {i+1}/{len(test_questions)}:")
    ask_question(question)

    # Delay to avoid rate limits except after last question
    if i < len(test_questions) - 1:
        print("Waiting 5 seconds before next question...")
        time.sleep(5)

print("\n" + "=" * 60)
print("Testing completed!")


Testing the Agentic RAG System...

Test 1/2:

Question: What was the total revenue for the fourth quarter of fiscal 2025, and what was the year-over-year growth percentage?
Answer: The total revenue for the fourth quarter of fiscal 2025 was $102.5 billion, and it represented an 8% year-over-year increase.
Waiting 5 seconds before next question...

Test 2/2:

Question: What is Apple's revenue growth forecast for the December quarter, and what is the primary reason for the expected strong performance in iPhone sales?
Answer: Apple's revenue growth forecast for the December quarter is expected to be 10% to 12% year-over-year, which will be the company's best quarter ever. 

The primary reason for the expected strong performance in iPhone sales is the double-digit year-over-year growth, which would be the company's best iPhone quarter ever.

Testing completed!


# **Agent 3: Stock Report Generator**

In [17]:
!pip install yfinance -q

In [27]:
import os
import pandas as pd
from datetime import datetime
import yfinance as yf
from groq import Groq

# Setup Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

### Example: Compare multiple stocks  

In [26]:
print("\nCOMPARING CHIP COMPANIES")
print("=" * 50)

chip_stocks = ["NVDA", "AMD", "INTC"]
chip_data = {}

for ticker in chip_stocks:
    # Get stock data for each chip company
    stock = yf.Ticker(ticker)
    info = stock.info

    current_price = info.get("currentPrice", "N/A")
    pe_ratio = info.get("trailingPE", "N/A")
    market_cap = info.get("marketCap", "N/A")
    week_high = info.get("fiftyTwoWeekHigh", "N/A")
    week_low = info.get("fiftyTwoWeekLow", "N/A")

    history_data = stock.history(period="6mo")
    avg_price = history_data['Close'].mean()
    price_volatility = history_data['Close'].std()

    news_items = []
    if hasattr(stock, "news") and stock.news:
        for item in stock.news[:3]:
            title = item.get("title", "No title available")
            news_items.append(title)
    else:
        news_items = ["No recent news"]

    chip_data[ticker] = {
        "current_price": current_price,
        "pe_ratio": pe_ratio,
        "market_cap": market_cap,
        "week_high": week_high,
        "week_low": week_low,
        "avg_price_6mo": avg_price,
        "price_volatility": price_volatility,
        "news": news_items
    }
    print(f"Loaded data for {ticker}")

# Multiple stocks comparison prompt
stock_info = ""
for ticker, stock_data in chip_data.items():
    stock_info += f"""
    {ticker}:
    - Price: ${stock_data.get('current_price', 'N/A')}
    - P/E Ratio: {stock_data.get('pe_ratio', 'N/A')}
    - Market Cap: {stock_data.get('market_cap', 'N/A')}
    - Recent News: {stock_data.get('news', ['N/A'])[0]}

    """

chip_prompt = f"""
Compare these stocks: {list(chip_data.keys())}

{stock_info}

Question: Which chip company has the best growth potential?

Please provide a comparison with:
1. Executive Summary
2. Key Differences
3. Recommendations
"""

# Get AI response for chip comparison
chip_response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": chip_prompt}],
    temperature=0.3
)

print(chip_response.choices[0].message.content)


COMPARING CHIP COMPANIES
Loaded data for NVDA
Loaded data for AMD
Loaded data for INTC
**Executive Summary**
The comparison of chip companies NVDA, AMD, and INTC reveals distinct differences in their current market performance and valuation. While all three companies are major players in the semiconductor industry, their growth potential can be assessed by examining their price, P/E ratio, market capitalization, and recent news. Based on the provided data, we can evaluate which company has the best growth potential.

**Key Differences**
The key differences between the three companies are:

* **Price**: AMD has the highest stock price at $259.65, followed by NVDA at $206.88, and INTC at $39.5.
* **P/E Ratio**: INTC has an unusually high P/E ratio of 1316.67, indicating potentially overvalued stock. AMD's P/E ratio is 156.42, and NVDA's is 58.77.
* **Market Capitalization**: NVDA has the largest market capitalization at $5.03 trillion, followed by AMD at $421 billion, and INTC at $188 b

### **Agent 4: LLM-as-a-Judge (Evaluate RAG Answers)**

In [35]:
# ========================================
# STEP 5: LLM-as-a-Judge (Evaluate RAG Answers)
# ========================================
!pip install -q groq

from groq import Groq
from textwrap import dedent
import json

# -------------------------------------------------
# 1. CONFIG – paste your Groq key
# -------------------------------------------------
GROQ_API_KEY = "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"   # <-- your key

client = Groq(api_key=GROQ_API_KEY)

# -------------------------------------------------
# 2. Evaluator function (no Agent class needed)
# -------------------------------------------------
def llm_judge(query: str, answer: str, context: str = "") -> str:
    """
    Sends query + answer + (optional) context to Groq and returns a markdown evaluation.
    """
    system_prompt = dedent("""\
    You are an impartial RAG evaluator. Rate the answer on a 1-5 scale for each criterion:

    1. **Faithfulness** – Does the answer stay true to the context (no hallucinations)?
    2. **Context Relevance** – How well does the answer use the provided context?
    3. **Answer Completeness** – Does it fully address the query?
    4. **Clarity** – Is the answer easy to read and well-structured?

    Respond **only** in the following markdown format:

    # Evaluation

    ## Scores
    - Faithfulness: X/5
    - Context Relevance: X/5
    - Answer Completeness: X/5
    - Clarity: X/5

    ## Overall: X/20
    ## Suggestions: ...
    """)

    user_prompt = dedent(f"""
    **QUERY:** {query}

    **CONTEXT:**
    {context or "No context provided."}

    **ANSWER:**
    {answer}
    """)

    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",      # fast & cheap
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_prompt}
        ],
        temperature=0.2,
        max_tokens=600
    )
    return completion.choices[0].message.content

# -------------------------------------------------
# 3. Helper to run evaluation (streaming optional)
# -------------------------------------------------
def evaluate_rag(query: str, answer: str, context_docs: list = None):
    """
    context_docs: list of dicts with at least 'page_content' (or just strings)
    """
    if context_docs:
        context = "\n---\n".join(
            [doc.get("page_content", doc)[:800] for doc in context_docs]
        )
    else:
        context = ""

    print("\n" + "="*60)
    print("EVALUATING RAG OUTPUT")
    print("="*60)
    print(f"QUERY: {query}\n")
    print(f"ANSWER: {answer}\n")
    if context:
        print(f"CONTEXT (first 300 chars): {context[:300]}…\n")
    else:
        print("CONTEXT: None\n")

    result = llm_judge(query, answer, context)
    print(result)

# -------------------------------------------------
# 4. Stock Trading Specific Example
# -------------------------------------------------
evaluate_rag(
    query="Should I invest in NVIDIA based on current market conditions?",
    answer="Based on the latest earnings report, NVIDIA shows strong growth potential with a 265% year-over-year revenue increase in Q1 2024. The company's data center segment grew by 427% due to AI chip demand. However, the stock is trading near its 52-week high of $140, which presents some short-term risk. I recommend a long-term position with dollar-cost averaging.",
    context_docs=[
        {"page_content": "NVIDIA Q1 2024 Earnings Report: Revenue reached $26 billion, up 265% year-over-year. Data center revenue was $22.6 billion, up 427% from previous year. Gaming revenue was $2.6 billion. Gross margin expanded to 78.9%."},
        {"page_content": "NVIDIA stock price analysis: Current price $128.50, 52-week high $140.20, 52-week low $39.23. P/E ratio 75.2. Market cap $3.1 trillion. The stock has gained 210% in the past year."},
        {"page_content": "Market conditions: Fed interest rates at 5.5%, inflation at 3.2%. AI sector expected to grow at 38% CAGR through 2030. Semiconductor industry facing supply chain constraints but demand remains strong."},
        {"page_content": "Analyst ratings for NVDA: 85% Buy, 12% Hold, 3% Sell. Average price target $145.50. Morgan Stanley raised target to $160 citing continued AI dominance. Goldman Sachs maintains Buy rating with $155 target."}
    ]
)


EVALUATING RAG OUTPUT
QUERY: Should I invest in NVIDIA based on current market conditions?

ANSWER: Based on the latest earnings report, NVIDIA shows strong growth potential with a 265% year-over-year revenue increase in Q1 2024. The company's data center segment grew by 427% due to AI chip demand. However, the stock is trading near its 52-week high of $140, which presents some short-term risk. I recommend a long-term position with dollar-cost averaging.

CONTEXT (first 300 chars): NVIDIA Q1 2024 Earnings Report: Revenue reached $26 billion, up 265% year-over-year. Data center revenue was $22.6 billion, up 427% from previous year. Gaming revenue was $2.6 billion. Gross margin expanded to 78.9%.
---
NVIDIA stock price analysis: Current price $128.50, 52-week high $140.20, 52-we…

# Evaluation

## Scores
- Faithfulness: 5/5
- Context Relevance: 5/5
- Answer Completeness: 4/5
- Clarity: 5/5

## Overall: 19/20
## Suggestions: Consider providing more information about potential risks and c